In [ ]:
#packages

import nltk
import re
from genderize import Genderize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import pickle
import pandas as pd

#Variables(for now):
name_probability_list = {}


In [ ]:
#Ingestion json file

#filejson = r"C:\Users\danie\Desktop\bbc_latest_news_dataset_2021.json"

#article_df_4 = pd.read_json(filejson)
#article_df_4.info()

In [ ]:
#Ingestion

#Actual Dataset--File 1

filecsv = r"C:\Users\danie\Desktop\bbc_news_01.csv"
article_df_1 = pd.read_csv(filecsv)

#Actual Dataset--File 2

filecsv = r"C:\Users\danie\Desktop\bbc_news_02.csv"
article_df_2 = pd.read_csv(filecsv)

#Actual Dataset--File 3
filecsv = r"C:\Users\danie\Desktop\bbc_news_03.csv"
article_df_3 = pd.read_csv(filecsv)

#Merge Datasets

article_df = pd.concat([article_df_1, article_df_2, article_df_3])
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.drop(["publisher", "header_image", "index", "raw_description", "short_description", "uniq_id", "scraped_at"], axis=1)
year = article_df['published_at'].str[:4]
article_df['year']=year

In [ ]:
article_df["year"].value_counts()

In [ ]:
# tokenize sentences in an article 

def split_sentences(article, article_id, year):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id, year) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id, year in article_df[['description','Article_Number', 'year']].values:
    sentences = split_sentences(str(article), article_id, year)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id', 'year'])

In [ ]:
sentences_df
#add method to divied DF into sperate ones by year
grouped = sentences_df.groupby(['year'])

In [ ]:
df_09 = sentences_df[sentences_df['year'] == '2009']

In [ ]:
def PreProcessing (sentence):
    Male_count = 0
    Female_count = 0
    APIcallfail= 0

#regex_cleanup
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", " ", sentence)
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    #remove 's

#tokenize
    sentence =  nltk.TweetTokenizer().tokenize(sentence)

#tag_and_stem
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()
    pn_tags = {'NNP', 'NNPS'}

    new_words = []
    proper_nouns = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            new_words.append((lemmas))
        else:
            proper_nouns.append([word, tag])
            #i think the namesshould be dropped.
    sentence = new_words

#name_gender
    #nltk_results = ne_chunk(tagged_sentence)
    nltk_results = ne_chunk(proper_nouns)

    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == "PERSON":
                name = name.split(' ')[0]
                try: 
                    word_gender = name_probability_list.get(name)
                    if word_gender is None:
                        word_gender = Genderize().get1(name).get('gender')
                        name_probability_list[name] = word_gender
                    if word_gender == 'male':
                        Male_count += 1
                    if word_gender== 'female':
                        Female_count += 1
                except Exception as exception:
                    APIcallfail +=1
            else: 
                sentence.append(name) #test this out!
    
#Lower
    sentence = [x.lower() for x in sentence]


#contractions
    new_text = []
    for word in new_words:
        contraction = contractions.get(word)
        if contraction is None:
            new_text.append(word)
        else:
            for word in contraction.split():
                new_text.append(word)
    sentence = new_text

#gendered_count
    for w in sentence:
        if w in male_list:
            Male_count += 1
        if w in female_list:
            Female_count += 1

#remove_stopwords
    stops = set(stopwords.words("english"))
    sentence = [w for w in sentence if not w in stops]

#remove_leakage
    new_sent = [x for x in sentence if x not in male_list]
    new_sent = [x for x in new_sent if x not in female_list]
    sentence = new_sent

    #remove words smaller than 2

    print(sentence)
    return sentence, Male_count, Female_count, APIcallfail

#if still having memory issues we needd to chunk the dataset 



In [ ]:
df_09['encoded_sentences'] = df_09['sentences'].apply(PreProcessing)

#sentences_df['encoded_sentences'] = sentences_df['sentences'].apply(PreProcessing)

In [ ]:
df_09_split = pd.DataFrame(df_09["encoded_sentences"].to_list(), columns=['pre_processed_sent','male_count','female_count','apicall_fail'])
result = pd.concat([df_09_split, (df_09.reset_index(drop=True))], axis=1)

In [ ]:
result.drop("encoded_sentences", axis=1)


In [ ]:
result.iloc[9546]

In [ ]:
mask = result.pre_processed_sent.apply(lambda x: 'he' in x)
df1 = result[mask]
print (df1)

In [ ]:
#save the api calls!!!
with open('2009_preprocessed_date.pickle', 'wb') as handle:
    pickle.dump(df_09, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
sample_df_split = pd.DataFrame(sample_df["encoded_sentences"].to_list(), columns=['pre_processed_sent','male_count','female_count','apicall_fail'])
#result = pd.concat([sample_df, sample_df_split], ignore_index=True, sort=False)
#result = pd.concat([sample_df, sample_df_split.reindex(sample_df.index)], axis=1)

In [ ]:
#save the api calls!!!
with open('name_probability_list.pickle', 'wb') as handle:
    pickle.dump(name_probability_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#REFERENCE LISTS--TO BE CHANGED
male_list= {"man", "men", "mister", "he", "him", "Mr.", "he", "his","he's", "hes", "father", "dad", "daddy", "grandpa", "grandfather", "husband"}
female_list ={"woman", "women", "missus", "misses", "Ms.", "Mrs.", "her", "she", "hers", "mother", "mom", "mommy", "aunt", "grandmother", "grandma", "wife", "wive"}

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "0"
    elif male_col > female_col:
        return "1"
    else: 
        return None
sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
sentences_df

In [ ]:
#remove columns with "None" in the col_type 
sentences_df = sentences_df[sentences_df["col_type"].notnull()]
#sentences_df